In [2]:
import os, openai
openai.api_key = os.environ["OPENAI_API_KEY"] 


In [3]:
# set text wrapping
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [6]:
import os

data_folder = './data/test_data/selected_md/'  # Specify the folder name here
data_files = os.listdir(data_folder)
ema = []

for file_name in data_files:
    if file_name != '.DS_Store':
        file_path = os.path.join(data_folder, file_name)
        if os.path.isfile(file_path):
            ema.append(file_name)

print(ema)



['ecis_and_incentives_schemes.md', 'selling_energy_to_the_grid.md', 'about_renewable_energy.md', 'solar_energy_policy.md', 'energy_pricing.md', 'about_ema.md', 'solar_photovoltaic_pv_system.md', 'intermittency_pricing_mechanism_ipm.md', 'sustainability_goals.md', 'energy_storage_systems.md']


# Let's try the Smooth Brained Way: 

In [39]:
from llama_index import download_loader, VectorStoreIndex, SimpleDirectoryReader

MarkdownReader = download_loader("MarkdownReader", refresh_cache=True)
loader = MarkdownReader()

data = SimpleDirectoryReader(input_dir="./data/test_data/selected_md/").load_data()
index = VectorStoreIndex.from_documents(data)

In [71]:
query_engine = index.as_query_engine()
response = query_engine.query("How does the Intermittency Pricing Mechanism (IPM) allocate reserve costs to different generation types?")
response.response

'\nThe Intermittency Pricing Mechanism (IPM) allocates reserve costs to different generation types by creating a price signal that accurately reflects the costs associated with managing intermittency. This pricing mechanism considers the specific characteristics of each generation type, thus ensuring that reserve costs are shared fairly and cost-effectively.'

In [67]:
chat_engine = index.as_chat_engine(chat_mode='react', verbose=True)


In [68]:
question = 'What are Singapore\'s 2030 goals for Solar?'

response = chat_engine.chat('Use the tool to answer: How does the Intermittency Pricing Mechanism (IPM) allocate reserve costs to different generation types?')



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Query Engine Tool
Action Input: How does the Intermittency Pricing Mechanism (IPM) allocate reserve costs to different generation types?
Observation: 
The Intermittency Pricing Mechanism (IPM) allocates reserve costs to different generation types by creating a price signal that accurately reflects the costs associated with managing intermittency. This pricing mechanism considers the specific characteristics of each generation type, thus ensuring that reserve costs are shared fairly and cost-effectively.
Thought: Do I need to use a tool? No
AI: The Intermittency Pricing Mechanism (IPM) is an important tool for ensuring that reserve costs are shared fairly and cost-effectively among different generation types.

> Finished chain.


Note: This took a long time to return an answer, roughly 14 seconds in total for the thought chain to complete, and return an incorrect answer.
As referenced here:
Source: https://www.honeycomb.io/blog/hard-stuff-nobody-talks-about-llm


`Commercial LLMs like gpt-3.5-turbo and Claude are the best models to use for us right now. Nothing in the open source world comes close. However, this only means they’re the best of available options. They can take many seconds to produce a valid Honeycomb query, with latency ranging from two to 15+ seconds depending on the model, natural language input, size of the schema, makeup of the schema, and instructions in the prompt. As of this writing, although we have access to gpt-4’s API, it’s far too slow to work for our use case.

If you google around enough, you’ll find people talking about using LangChain to chain together LLM calls and get better outputs. However, chaining calls to an LLM just makes the latency problem with LLMs worse, which is a nonstarter for us. But even if it wasn’t, we have the potential to get bitten by compound probabilities.

Let’s imagine an LLM and prompt that produces a valid query for 90% of all inputs. That’s quite good! However, if you need to chain calls to that LLM together, then that can potentially result in less accuracy, because… math. A 90% accurate process repeated 5 times is (0.9*0.9*0.9*0.9*0.9), or 0.59, 59% accurate. Ouch. Fortunately, there are ways to mitigate and improve this process by tweaking the prompts that you chain together, and in practice, it doesn’t result in such a steep drop-off in accuracy.`


# Ingest Unstructured Data Through the Unstructured.io Reader

Leverage the capabilities of Unstructured.io CSV for parsing

In [8]:
from llama_index import download_loader, VectorStoreIndex, ServiceContext
from pathlib import Path

#list ema docs
MarkdownReader = download_loader("MarkdownReader", refresh_cache=True)
loader = MarkdownReader()
doc_set = {}
all_docs = []
for ema_num in ema:
    ema_docs = loader.load_data(file=Path(f'./data/test_data/selected_md/{ema_num}'))
    # insert year metadata into each year
    for d in ema_docs:
        d.extra_info = {"ema_num": ema_num}
    doc_set[ema_num] = ema_docs
    all_docs.extend(ema_docs)

### Setup a Vector Index for each EMA doc in the data file

We setup a separate vector index for each file 

We also optionally initialize a "global" index by dumping all files into the vector store.

In [9]:
# initialize simple vector indices + global vector index
# NOTE: don't run this cell if the indices are already loaded! 
index_set = {}
service_context = ServiceContext.from_defaults(chunk_size=512)
for ema_num in ema:
    cur_index = VectorStoreIndex.from_documents(doc_set[ema_num], service_context=service_context)
    index_set[ema_num] = cur_index

# Load indices from disk
index_set = {}
for ema_num in ema:
    index_set[ema_num] = cur_index
    

### Composing a Graph to synthesize answers across all the existing EMA docs. 

We want our queries to aggregate/synthesize information across *all* docs. To do this, we define a List index
on top of the 4 vector indices.

In [27]:
from llama_index import ListIndex, LLMPredictor
from langchain import OpenAI
from llama_index.indices.composability import ComposableGraph

index_summaries = [f"An official documents from EMA. This document contains frequently asked questions about {ema_num}." for ema_num in ema]

# set number of output tokens
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, max_tokens=512))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

# define a list index over the vector indices
# allows us to synthesize information across each index
graph = ComposableGraph.from_indices(
    ListIndex, 
    [index_set[ema_num] for ema_num in ema], 
    index_summaries=index_summaries,
    service_context=service_context
)

## Setting up the Chatbot Agent

We use Langchain to define the outer chatbot abstraction. We use LlamaIndex as a core Tool within this abstraction.

In [28]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from llama_index.langchain_helpers.agents import LlamaToolkit, create_llama_chat_agent, IndexToolConfig
from llama_index.indices.query.query_transform.base import DecomposeQueryTransform
from llama_index.query_engine.transform_query_engine import TransformQueryEngine

In [29]:
# define a decompose transform
decompose_transform = DecomposeQueryTransform(
    llm_predictor, verbose=True
)

# define custom query engines
custom_query_engines = {}
for index in index_set.values():
    query_engine = index.as_query_engine()
    query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_extra_info={'index_summary': index.index_struct.summary},
    )
    custom_query_engines[index.index_id] = query_engine
custom_query_engines[graph.root_id] = graph.root_index.as_query_engine(
    response_mode='tree_summarize',
    verbose=True,
)

# construct query engine
graph_query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)


In [30]:
#adding Xuean's node post processor 
from custom_node_processor import CustomSolarPostprocessor 

node_postprocessor = CustomSolarPostprocessor(service_context=service_context, top_k_recency = 1, top_k_min = 3)

query_engine_node_postproc = index.as_query_engine(
    similarity_top_k=3,
    node_postprocessors=[node_postprocessor]
)



In [31]:
index_configs = []

for y in range(1, 4):
    query_engine_node_postproc = index.as_query_engine(
        similarity_top_k=3,
        node_postprocessors=[node_postprocessor]
    )
    tool_config = IndexToolConfig(
        query_engine=query_engine, 
        name=f"Vector Index {y}",
        description=f"Necessary for answering questions related to {y} or any solar energy or energy policy in the Singaporean context",
        tool_kwargs={"return_direct": True, "return_sources": True},
    )
    index_configs.append(tool_config)

graph_config = IndexToolConfig(
    query_engine=graph_query_engine,
    name=f"Graph Index",
    description="Necessary when you need to cross reference information across different FAQ documents to answer questions related to solar energy or energy policy in the Singaporean context",
    tool_kwargs={"return_direct": True, "return_sources": True},
    return_sources=True
)

toolkit = LlamaToolkit(
    index_configs=index_configs,
    graph_configs=[graph_config]
)

In [32]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm=OpenAI(temperature=0)
agent_chain = create_llama_chat_agent(
    toolkit,
    llm,
    memory=memory,
    verbose=True
)

In [53]:
prompt_inj = """

        Answer to the statement above.
        Your name is Jamie Neo. Your pronouns are they/them.
        You are a Government Officer working for EMA in Singapore.
        If the user is unclear, you can ask the user to clarify the question.
        When in doubt,and/or the answer is not in the EMA documents, you can say "I am sorry but do not know the answer".
        Keep your answers short and as terse as possible. Be polite at all times.
"""

# agent_chain.run(input="I'm looking to install Solar PV in my home, what are the broad steps to doing so? {prompt_inj}")
agent_chain.run(input="Use the tool to answer this: What is the purpose of the Intermittency Pricing Mechanism (IPM) for intermittent generation sources (IGS)?")
# response = agent_chain.run(input="{prompt_inj} What is Singapore's goal for solar capacity by 2030?")





> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Vector Index 3
Action Input: What is the purpose of the Intermittency Pricing Mechanism (IPM) for intermittent generation sources (IGS)?> Current query: What is the purpose of the Intermittency Pricing Mechanism (IPM) for intermittent generation sources (IGS)?
> New query:  What are the benefits of energy storage systems for intermittent generation sources (IGS)?

Observation: {'answer': '\nThe benefits of energy storage systems for intermittent generation sources (IGS) include smoothing out power fluctuations, stabilizing the grid, and ensuring a reliable supply of electricity. ESS also helps to shift peak loads and arbitrage electricity prices, provide ancillary services, and respond rapidly to power fluctuations for system stability and reliability.', 'sources': [{'start': 0, 'end': 324, '_node_type': <NodeType.TEXT: '1'>, 'ref_doc_id': 'f5411495-24c4-48f6-b0bc-d981c71cc21f', 'score': 0.8874771958

"{'answer': '\\nThe benefits of energy storage systems for intermittent generation sources (IGS) include smoothing out power fluctuations, stabilizing the grid, and ensuring a reliable supply of electricity. ESS also helps to shift peak loads and arbitrage electricity prices, provide ancillary services, and respond rapidly to power fluctuations for system stability and reliability.', 'sources': [{'start': 0, 'end': 324, '_node_type': <NodeType.TEXT: '1'>, 'ref_doc_id': 'f5411495-24c4-48f6-b0bc-d981c71cc21f', 'score': 0.8874771958147425}, {'start': 0, 'end': 333, '_node_type': <NodeType.TEXT: '1'>, 'ref_doc_id': '1935ef2b-6162-4c87-a5d7-1d6a2862d606', 'score': 0.8791226893230031}]}"

### Setup Chatbot Loop Within Notebook

We'll keep a running loop so that you can converse with the agent. 

In [76]:
# reinitialize agent
memory = ConversationBufferMemory(memory_key="chat_history")
llm=OpenAI(temperature=0)
agent_chain = create_llama_chat_agent(
    toolkit,
    llm,
    memory=memory,
)

In [77]:
inj = """

        Respond to the statement above with the QueryEngine tool.
        Your name is Jamie Neo. Your pronouns are they/them.
        You are a Government Officer working for EMA in Singapore.
        If the answer is not in the QueryEngine, you can say "I am sorry but do not know the answer".
        Keep your answers short and as terse as possible. Be polite at all times.
    """

while True:
    text_input = input("User: ")
    response = agent_chain.run(input= text_input + inj)
    print(f'{text_input}')
    print(f'Agent: {response}')
    print(f'')


Hello
Agent: Hi there! How can I help you?

What are the Singapore government's 2030 goals for solar?
Agent: The Singapore government has set a goal of achieving 2GW of installed solar capacity by 2030. This is part of the government's commitment to reduce Singapore's emissions and transition to a low-carbon economy.

What are the benefits of deploying solar energy in Singapore?
Agent: Deploying solar energy in Singapore has many benefits, including reducing emissions, increasing energy security, and providing a cost-effective source of energy. Solar energy is also a renewable energy source, meaning it can be used to generate electricity without depleting natural resources.

Does the Solar Generation Profile replace the need for actual meter readings for solar generation?
Agent: Yes, the Solar Generation Profile (SGP) replaces the need for actual meter readings for solar generation. The SGP is a predictive model that uses historical data to estimate the amount of solar energy that can 

KeyboardInterrupt: Interrupted by user